In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import hamming_loss, jaccard_score, f1_score, precision_score, recall_score, accuracy_score

In [2]:
data = pd.read_csv('veri_seti.csv',sep=",")
df = pd.DataFrame(data)

In [3]:
df

,metin,kelimeler,departman
0,"Kahramanmaraş'ta çöp toplama sık sık aksıyor,m...","['çöp toplama','koku']",Çevre Koruma ve Sıfır Atık Dairesi Başkanlığı
1,"Yol yapım çalışmaları çok yavaş ilerliyor,tozd...","['yol yapım','toz']",Yol Yapım Bakım ve Onarım Dairesi Başkanlığı
2,Parklarda yeterli bakım yapılmıyor.,['park bakımı'],Park ve Bahçeler Müdürlüğü
3,"Ulaşım araçları çok kalabalık,otobüs durakları...",['ulaşım'],Ulaşım Dairesi Başkanlığı
4,"Kışın kar temizleme çalışmaları yetersiz,yolla...","['kar temizleme','yol']",Yol Yapım Bakım ve Onarım Dairesi Başkanlığı
...,...,...,...
271,"Şehirde su kalitesi düşük, içme suyu sağlıksız.",['su kalitesi'],Çevre Koruma ve Sıfır Atık Dairesi Başkanlığı
272,"Mahallemizde park yerleri aydınlatmasız, gece ...",['park yeri aydınlatması'],Fen İşleri Dairesi Başkanlığı
273,"Şehirde atık su arıtma tesisleri yetersiz, çev...","['atık su arıtma','çevre kirliliği']",Çevre Koruma ve Sıfır Atık Dairesi Başkanlığı
274,Mahallemizde otobüs duraklarında bilgilendirme...,['bilgilendirme panoları'],Ulaşım Dairesi Başkanlığı


In [4]:
def preprocess(data):
    processed_data = []
    for textt in data["metin"]:
        processed_text = re.sub('[^a-zA-ZÇçşŞıİğĞöÖüÜ]', ' ', textt).lower()
        words = processed_text.split()
        processed_data.append(' '.join(words))
    return processed_data

print(preprocess(data))

['kahramanmaraş ta çöp toplama sık sık aksıyor mahallemiz kokudan geçilmiyor', 'yol yapım çalışmaları çok yavaş ilerliyor tozdan geçilmiyor', 'parklarda yeterli bakım yapılmıyor', 'ulaşım araçları çok kalabalık otobüs duraklarında uzun süre bekliyoruz', 'kışın kar temizleme çalışmaları yetersiz yollar çok geç açılıyor', 'belediye parklarında güvenlik sorunu var huzursuz hissediyoruz', 'belediye otobüslerinin sefer saatleri düzenli değil', 'çevre temizliği konusunda yetersiz kalınıyor özellikle piknik alanları çok kirli', 'kahramanmaraş ta yeşil alanlar yetersiz', 'belediye web sitesi kullanışlı değil bilgiye ulaşmak zor', 'kahramanmaraş ta kültürel etkinlikler yetersiz', 'son zamanlarda şehrin birçok noktasında çöp konteynerlerinin yetersiz olması ve çöplerin zamanında toplanmaması nedeniyle çevre kirliliği ciddi bir sorun haline geldi özellikle sıcak havalarda kötü kokular ve sinek istilası yaşanıyor belediyenin bu konuda daha sıkı önlemler alması gerekiyor', 'yaşadığım mahallede bulu

In [5]:
# Daha kapsamlı bir noktalama işareti kümesi(Türkçe için)
punctuations = string.punctuation + "’"

# Stop kelimeler
stop_words = list(stopwords.words('turkish'))

def custom_tokenize(text):
    # Daha kapsamlı kesme işareti kuralı
    text = re.sub(r"(\w)'(\w)|(\w)'", r"\1'\2\3", text) #Kelimeler arasındaki kesme işaretlerini belirli kurallara göre düzenler.
    text = re.sub(r"(\w)'", r"\1", text)  # Kesme işaretini kaldırır
    # Metni küçük harflere çevir
    text = text.lower()
    # Tokenleştirme
    words = word_tokenize(text)
    # Noktalama işaretlerini küme kullanarak hızlıca kaldır
    punctuation_set = set(punctuations)
    words = [word for word in words if word not in punctuation_set]
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

for text in data["metin"]:
    words = custom_tokenize(text)
    print(words)

kahramanmaraşta çöp toplama sık sık aksıyor mahallemiz kokudan geçilmiyor
yol yapım çalışmaları yavaş ilerliyor tozdan geçilmiyor
parklarda yeterli bakım yapılmıyor
ulaşım araçları kalabalık otobüs duraklarında uzun süre bekliyoruz
kışın kar temizleme çalışmaları yetersiz yollar geç açılıyor
belediye parklarında güvenlik sorunu var huzursuz hissediyoruz
belediye otobüslerinin sefer saatleri düzenli değil
çevre temizliği konusunda yetersiz kalınıyor özellikle piknik alanları kirli
kahramanmaraşta yeşil alanlar yetersiz
belediye web sitesi kullanışlı değil bilgiye ulaşmak zor
kahramanmaraşta kültürel etkinlikler yetersiz
son zamanlarda şehrin birçok noktasında çöp konteynerlerinin yetersiz olması çöplerin zamanında toplanmaması nedeniyle çevre kirliliği ciddi bir sorun haline geldi.özellikle sıcak havalarda kötü kokular sinek istilası yaşanıyor.belediyenin konuda sıkı önlemler alması gerekiyor
yaşadığım mahallede bulunan parkın bakımsız olması oyun gruplarının güvensiz olması nedeniyle ç

In [6]:
df['processed_text'] = (df['metin'].apply(custom_tokenize) + df["kelimeler"].apply(custom_tokenize))
print(df["processed_text"])

0      kahramanmaraşta çöp toplama sık sık aksıyor ma...
1      yol yapım çalışmaları yavaş ilerliyor tozdan g...
2         parklarda yeterli bakım yapılmıyor'park bakımı
3      ulaşım araçları kalabalık otobüs duraklarında ...
4      kışın kar temizleme çalışmaları yetersiz yolla...
                             ...                        
271    şehirde su kalitesi düşük içme suyu sağlıksız'...
272    mahallemizde park yerleri aydınlatmasız gece k...
273    şehirde atık su arıtma tesisleri yetersiz çevr...
274    mahallemizde otobüs duraklarında bilgilendirme...
275    belediye etkinliklerinde engelli vatandaşlar d...
Name: processed_text, Length: 276, dtype: object


In [7]:
# TF-IDF Vektörizasyon(Metin verilerini sayısal bir formata dönüştürme)
tfidf = TfidfVectorizer(max_features=1000, min_df=1, stop_words=stop_words, ngram_range=(1,2))
X = tfidf.fit_transform(df['processed_text'])

In [8]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
y = encoder.fit_transform(df[['departman']])   #One-Hot Encoding,bu tür verileri sayısal bir formata dönüştürmek için kullanılan bir tekniktir.

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer
# Veri temizleme
df['departman'] = df['departman'].astype(str).str.lower().str.strip()

# Çoklu etiket dönüşümü
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['departman'].apply(lambda x: x.split(', ')))

# Oluşturulan etiketleri görselleştirme
print(mlb.classes_)  # Tüm etiketleri gösterir

['"fen i̇şleri dairesi başkanlığı"' '"ulaşım dairesi başkanlığı"'
 '"çevre koruma ve sıfır atık dairesi başkanlığı"'
 'basın yayın ve halkla i̇lişkiler dairesi başkanlığı'
 'bilgi i̇şlem dairesi başkanlığı' 'destek hizmetleri dairesi başkanlığı'
 'dış i̇lişkiler dairesi başkanlığı' 'fen i̇şleri dairesi başkanlığı'
 'i̇mar ve şehircilik dairesi başkanlığı'
 'i̇nsan kaynakları ve eğitim dairesi başkanlığı'
 'i̇tfaiye ve afet i̇şleri dairesi başkanlığı'
 'kadın aile ve sosyal hizmetler dairesi başkanlığı'
 'kuzey i̇lçeleri dairesi başkanlığı'
 'kültür gençlik ve spor dairesi başkanlığı'
 'mali hizmetler dairesi başkanlığı' 'park ve bahçeler müdürlüğü'
 'tarımsal hizmetler ve muhtarlıklar dairesi başkanlığı'
 'ulaşım dairesi başkanlığı'
 'yol yapım bakım ve onarım dairesi başkanlığı'
 'zabıta dairesi başkanlığı'
 'çevre koruma ve sıfır atık dairesi başkanlığı']


In [10]:
# Departman sütunundaki tekrar eden değerleri kaldırmak
unique_departments = set(df['departman'].dropna())

# Set'i listeye dönüştürme (isteğe bağlı)
unique_departments_list = list(unique_departments)

print(unique_departments_list)

['basın yayın ve halkla i̇lişkiler dairesi başkanlığı', 'yol yapım bakım ve onarım dairesi başkanlığı', 'i̇mar ve şehircilik dairesi başkanlığı', 'çevre koruma ve sıfır atık dairesi başkanlığı', 'bilgi i̇şlem dairesi başkanlığı', 'tarımsal hizmetler ve muhtarlıklar dairesi başkanlığı', 'park ve bahçeler müdürlüğü', 'i̇tfaiye ve afet i̇şleri dairesi başkanlığı', 'fen i̇şleri dairesi başkanlığı', 'zabıta dairesi başkanlığı', 'dış i̇lişkiler dairesi başkanlığı', 'destek hizmetleri dairesi başkanlığı', 'mali hizmetler dairesi başkanlığı', 'kültür gençlik ve spor dairesi başkanlığı', 'i̇nsan kaynakları ve eğitim dairesi başkanlığı', 'ulaşım dairesi başkanlığı', 'kuzey i̇lçeleri dairesi başkanlığı', '"ulaşım dairesi başkanlığı"', 'kadın aile ve sosyal hizmetler dairesi başkanlığı', '"fen i̇şleri dairesi başkanlığı"', '"çevre koruma ve sıfır atık dairesi başkanlığı"']


In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
# Model ve hiperparametre ayarları
param_dist = {
    'model__estimator__n_estimators': [50, 100, 200],
    'model__estimator__max_depth': [None, 10, 20, 30],
    'model__estimator__min_samples_split': [2, 5, 10],
    'model__estimator__max_features': ['sqrt', 'log2', None],
    'model__estimator__bootstrap': [True, False]
}
# Pipeline oluşturma
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('feature_selector', SelectKBest(score_func=chi2, k='all')),
    ('model', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

# Randomized Search ile hiperparametre optimizasyonu
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=20, cv=5, scoring='f1_samples', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# En iyi modeli seçin
best_model = random_search.best_estimator_

# Tahmin yapma
y_pred = best_model.predict(X_test)

# Performans değerlendirmesi
print("Hamming Loss:", hamming_loss(y_test, y_pred))
print("Jaccard Score:", jaccard_score(y_test, y_pred, average='samples'))
print("F1 Score:", f1_score(y_test, y_pred, average='samples'))
print("Precision Score:", precision_score(y_test, y_pred, average='samples'))
print("Recall Score:", recall_score(y_test, y_pred, average='samples'))
print("Doğruluk:", accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

Hamming Loss: 0.02891156462585034
Jaccard Score: 0.5178571428571429
F1 Score: 0.5357142857142857
Precision Score: 0.5178571428571429
Recall Score: 0.5714285714285714
Doğruluk: 0.5


c:\Users\faatm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
from collections import defaultdict
import ast

def create_keyword_dict(df):
    keyword_dict = defaultdict(list)
    
    def process_row(row):
        departman = row['departman']
        # `ast.literal_eval` ile string listeyi Python listesine dönüştür
        kelimeler = ast.literal_eval(row['kelimeler'])
        # Verilerin küçük harfe dönüştürülmesi
        kelimeler = [kelime.lower().strip() for kelime in kelimeler]
        keyword_dict[departman].extend(kelimeler)
    
    df.apply(process_row, axis=1)
    
    return keyword_dict

keyword_dict = create_keyword_dict(df)
print(keyword_dict)

defaultdict(<class 'list'>, {'çevre koruma ve sıfır atık dairesi başkanlığı': ['çöp toplama', 'koku', 'çevre temizliği', 'çöp toplama', 'çöp konteynerler', 'geri dönüşüm', 'temizlik personeli', 'çöp', 'sokak temizliği', 'çöp', 'çevre kirliliği', 'geri dönüşüm kutusu', 'çöp', 'çöp konteynerleri', 'tıbbi atıklar', 'geri dönüşüm', 'atık kağıtlar', 'temizlik', 'geri dönüşüm', 'sıfır atık projeleri', 'çevre kirliliği', 'geri dönüşüm kutuları', 'çöp konteyneri', 'koku', 'hava kirliliği', 'çöp konteyneri', 'çöpler', 'çöp konteynerleri', 'çevre temizliği', 'kir', 'çöp konteynerleri', 'geri dönüşüm kutuları', 'atıklar', 'hava kirliliği', 'çöp toplama saatleri', 'çöp konteynerleri', 'gürültü kirliliği', 'sokak temizliği', 'hava kalitesi', 'hava kirliliği', 'çöp toplama', 'su kalitesi', 'atık su arıtma', 'çevre kirliliği'], 'yol yapım bakım ve onarım dairesi başkanlığı': ['yol yapım', 'toz', 'kar temizleme', 'yol', 'kar temizliği', 'yol kaplama', 'yol onarımı', 'yol yapım projeleri', 'yol bakım',

In [14]:
def find_departman(metin, keyword_dict):
    for departman, kelime_listesi in keyword_dict.items():
        for kelime in kelime_listesi:
            if kelime in metin:
                return departman
    return None

In [15]:
# Kullanıcıdan metin alma
user_text = input("Lütfen Şikayetinizi Giriniz:")
cleaned_text = custom_tokenize(user_text)
print(cleaned_text)

# İlgili departmanı bulma
departman = find_departman(cleaned_text, keyword_dict)

# Sonucu ekrana yazdırma
if departman:
    print(f"Şikayetinizin ilgili departmanı: {departman}")
else:
    print("Şikayetinizle ilgili departman bulunamadı.")

kahramanmaraş web sitesi alakalı sorun yaşıyorum sorun bir an önce çözülsün lütfen
Şikayetinizin ilgili departmanı: bilgi i̇şlem dairesi başkanlığı
